In [ ]:
# 导入必要的库
import numpy as np  # 导入NumPy库，用于数值计算
import random  # 导入随机数生成库

# 数据采样设置
n = 40428967  # 点击流数据中的记录总数
sample_size = 1000000  # 需要采样的数据量（100万条记录）
skip_values = sorted(random.sample(range(1,n), n-sample_size))  # 创建要跳过的行索引列表，从而实现随机采样

In [ ]:
# 定义训练数据的列类型
types_train = {
    'id': np.dtype(int),           # 记录ID
    'click': np.dtype(int),        # 是否被点击（目标变量）
    'hour': np.dtype(int),         # 时间小时数
    'C1': np.dtype(int),           # 类别特征1
    'banner_pos': np.dtype(int),   # 横幅位置
    'site_id': np.dtype(str),      # 网站ID
    'site_domain': np.dtype(str),  # 网站域名 
    'site_category': np.dtype(str), # 网站类别
    'app_id': np.dtype(str),        # 应用ID
    'app_domain': np.dtype(str),    # 应用域名
    'app_category': np.dtype(str),  # 应用类别
    'device_id': np.dtype(str),     # 设备ID
    'device_ip': np.dtype(str),     # 设备IP地址
    'device_model': np.dtype(str),  # 设备型号
    'device_type': np.dtype(int),   # 设备类型
    'device_conn_type': np.dtype(int), # 设备连接类型
    'C14': np.dtype(int),           # 类别特征14
    'C15': np.dtype(int),           # 类别特征15
    'C16': np.dtype(int),           # 类别特征16
    'C17': np.dtype(int),           # 类别特征17
    'C18': np.dtype(int),           # 类别特征18
    'C19': np.dtype(int),           # 类别特征19
    'C20': np.dtype(int),           # 类别特征20
    'C21': np.dtype(int)            # 类别特征21
}


In [ ]:
types_test = {
    'id': np.dtype(int),           # 记录ID
    'hour': np.dtype(int),         # 时间小时数
    'C1': np.dtype(int),           # 类别特征1
    'banner_pos': np.dtype(int),   # 横幅位置
    'site_id': np.dtype(str),      # 网站ID
    'site_domain': np.dtype(str),  # 网站域名
    'site_category': np.dtype(str), # 网站类别
    'app_id': np.dtype(str),        # 应用ID
    'app_domain': np.dtype(str),    # 应用域名
    'app_category': np.dtype(str),  # 应用类别
    'device_id': np.dtype(str),     # 设备ID
    'device_ip': np.dtype(str),     # 设备IP地址
    'device_model': np.dtype(str),  # 设备型号
    'device_type': np.dtype(int),   # 设备类型
    'device_conn_type': np.dtype(int), # 设备连接类型
    'C14': np.dtype(int),           # 类别特征14
    'C15': np.dtype(int),           # 类别特征15
    'C16': np.dtype(int),           # 类别特征16
    'C17': np.dtype(int),           # 类别特征17
    'C18': np.dtype(int),           # 类别特征18
    'C19': np.dtype(int),           # 类别特征19
    'C20': np.dtype(int),           # 类别特征20
    'C21': np.dtype(int)            # 类别特征21
}

In [ ]:
import pandas as pd
import gzip

parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')

with gzip.open('train.gz') as f:
    train = pd.read_csv(f, parse_dates = ['hour'], date_parser = parse_date, dtype=types_train, skiprows = skip_values)

train.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x='click',data=train, palette='hls')
plt.show()

In [ ]:
train.groupby('hour').agg({'click':'sum'}).plot(figsize=(12,6))
plt.ylabel('Number of clicks')
plt.title('Number of clicks by hour')

In [ ]:
train.groupby(['hour_of_day', 'click']).size().unstack().plot(kind='bar', title="Hour of Day", figsize=(12,6))
plt.ylabel('count')
plt.title('Hourly impressions vs. clicks')

In [ ]:
import seaborn as sns

df_click = train[train['click'] == 1]
df_hour = train[['hour_of_day','click']].groupby(['hour_of_day']).count().reset_index()
df_hour = df_hour.rename(columns={'click': 'impressions'})
df_hour['clicks'] = df_click[['hour_of_day','click']].groupby(['hour_of_day']).count().reset_index()['click']
df_hour['CTR'] = df_hour['clicks']/df_hour['impressions']*100

plt.figure(figsize=(12,6))
sns.barplot(y='CTR', x='hour_of_day', data=df_hour)
plt.title('Hourly CTR');

In [ ]:
train['day_of_week'] = train['hour'].apply(lambda val: val.weekday_name)
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
train.groupby('day_of_week').agg({'click':'sum'}).reindex(cats).plot(figsize=(12,6))
ticks = list(range(0, 7, 1)) 
labels = "Mon Tues Weds Thurs Fri Sat Sun".split()
plt.xticks(ticks, labels)
plt.title('click trends by day of week');

In [ ]:
train.groupby(['day_of_week','click']).size().unstack().reindex(cats).plot(kind='bar', title="Day of the Week", figsize=(12,6))
ticks = list(range(0, 7, 1)) 
plt.xticks(ticks, labels)
plt.title('Impressions vs. clicks by day of week');

In [ ]:
df_click = train[train['click'] == 1]
df_dayofweek = train[['day_of_week','click']].groupby(['day_of_week']).count().reset_index()
df_dayofweek = df_dayofweek.rename(columns={'click': 'impressions'})
df_dayofweek['clicks'] = df_click[['day_of_week','click']].groupby(['day_of_week']).count().reset_index()['click']
df_dayofweek['CTR'] = df_dayofweek['clicks']/df_dayofweek['impressions']*100

plt.figure(figsize=(12,6))
sns.barplot(y='CTR', x='day_of_week', data=df_dayofweek, order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Day of week CTR')

In [ ]:
def convert_obj_to_int(self):
    """
    将DataFrame中的对象(字符串)类型列转换为整数类型
    通过计算每个值的哈希值实现转换，并删除原始列
    """
    # 获取所有列名和它们的数据类型
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    # 新列名后缀
    new_col_suffix = '_int'
    
    # 遍历所有列
    for index in range(0,len(object_list_columns)):
        # 检查列的数据类型是否为对象类型(通常是字符串)
        if object_list_dtypes[index] == object:
            # 创建新列，使用hash函数将对象值转换为整数
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map(lambda x: hash(x))
            # 删除原始的对象类型列
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    
    return self

In [ ]:
# 应用convert_obj_to_int函数处理训练数据，将所有对象类型列转换为整数类型
train = convert_obj_to_int(train)

train.drop('hour', axis=1, inplace=True)
train.drop('id', axis=1, inplace=True)

# 导入LightGBM库，这是一个高效的梯度提升决策树框架
import lightgbm as lgb

# 从训练数据中提取特征X，排除目标变量'click'
X_train = train.loc[:, train.columns != 'click']

# 提取目标变量y，即'click'列的值
y_target = train.click.values

# 创建训练集和验证集的划分掩码（80%训练，20%验证）
msk = np.random.rand(len(X_train)) < 0.8

# 使用80%的数据创建LightGBM训练数据集
lgb_train = lgb.Dataset(X_train[msk], y_target[msk])

# 使用剩余20%的数据创建LightGBM验证数据集
# reference=lgb_train确保验证集使用与训练集相同的特征映射
lgb_eval = lgb.Dataset(X_train[~msk], y_target[~msk], reference=lgb_train)

# 指定LightGBM的配置参数
params = {
    'task': 'train',                    # 任务类型：训练模式
    'boosting_type': 'gbdt',            # 提升类型：梯度提升决策树
    'objective': 'binary',              # 目标函数：二分类问题
    'metric': { 'binary_logloss'},      # 评估指标：二元对数损失
    'num_leaves': 31,                   # 每棵树的最大叶子数量（默认为31）
    'learning_rate': 0.08,              # 学习率：控制每棵树的贡献权重
    'feature_fraction': 0.7,            # 特征抽样比例：每次训练前随机选择70%的特征
    'bagging_fraction': 0.3,            # 数据抽样比例：每次迭代随机选择30%的数据进行训练
    'bagging_freq': 5,                  # 数据抽样频率：每5次迭代进行一次数据抽样
    'verbose': 0                         # 不显示训练过程的详细信息
}

print('Start training...')              # 打印训练开始提示
# 训练模型
gbm = lgb.train(params,                 # 模型参数
                lgb_train,              # 训练数据集
                num_boost_round=4000,   # 最大迭代次数：最多训练4000棵树
                valid_sets=lgb_eval,    # 验证数据集：用于监控训练进程
                early_stopping_rounds=500)  # 早停策略：如果验证集性能在500轮内没有提升，则停止训练

In [ ]:
# 导入必要的库
from operator import itemgetter  # 用于从对象中获取特定项目
from sklearn.model_selection import train_test_split  # 用于划分训练集和验证集
import xgboost as xgb  # 导入XGBoost库
from sklearn.metrics import roc_auc_score  # 用于计算ROC曲线下面积

def run_default_test(train, test, features, target, random_state=0):
    """
    使用XGBoost训练二分类模型的函数
    
    参数:
    train - 训练数据集
    test - 测试数据集
    features - 特征列名列表
    target - 目标变量列名
    random_state - 随机种子，用于复现结果
    """
    # 设置XGBoost模型的关键超参数
    eta = 0.1  # 学习率
    max_depth = 5  # 树的最大深度
    subsample = 0.8  # 样本采样比例
    colsample_bytree = 0.8  # 特征采样比例
    
    # 打印模型超参数信息
    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth, subsample, colsample_bytree))
    
    # 定义XGBoost模型参数
    params = {
        "objective": "binary:logistic",  # 二分类任务
        "booster" : "gbtree",  # 使用树模型作为基学习器
        "eval_metric": "logloss",  # 使用对数损失作为评估指标
        "eta": eta,  # 学习率
        "max_depth": max_depth,  # 树的最大深度
        "subsample": subsample,  # 训练每棵树时使用的样本比例
        "colsample_bytree": colsample_bytree,  # 训练每棵树时使用的特征比例
        "silent": 1,  # 不输出中间过程信息
        "seed": random_state  # 随机种子
    }
    
    num_boost_round = 260  # 最大迭代次数
    early_stopping_rounds = 20  # 早停轮数：如果验证集性能在20轮内没有提升则停止训练
    test_size = 0.2  # 验证集占比为20%

    # 将训练数据划分为训练集和验证集
    X_train, X_valid = train_test_split(train, test_size=test_size, random_state=random_state)
    
    # 提取训练集和验证集的目标变量
    y_train = X_train[target]
    y_valid = X_valid[target]
    
    # 创建XGBoost数据矩阵
    dtrain = xgb.DMatrix(X_train[features], y_train)
    dvalid = xgb.DMatrix(X_valid[features], y_valid)
    
    # 设置训练过程中需要监控的数据集
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    
    # 训练XGBoost模型
    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, 
                   early_stopping_rounds=early_stopping_rounds, verbose_eval=True)
                   
features = ['C1', 'banner_pos', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'hour_of_day',
       'site_id_int', 'site_domain_int', 'site_category_int', 'app_id_int',
       'app_domain_int', 'app_category_int', 'device_id_int', 'device_ip_int',
       'device_model_int', 'day_of_week_int']
run_default_test(train, y_target, features, 'click')